In [11]:
import cv2
import numpy as np


gesture = {
    0:'ㄱ', 1:'ㄴ', 2:'ㄷ' , 3:'ㄹ' , 4:'ㅁ' , 5:'ㅂ', 6:'ㅅ', 7:'ㅇ', 8:'ㅈ', 9:'ㅊ', 10:'ㅋ', 11:'ㅌ', 12:'ㅍ',
    13:'ㅎ',
}


# 모델 생성 (향후 분리할 필요 있음)
file = np.genfromtxt('dataSet/confine_data.csv', delimiter=',')
angle = file[:,:-1].astype(np.float32)
label = file[:, -1].astype(np.float32)
knn = cv2.ml.KNearest_create()
knn.train(angle, cv2.ml.ROW_SAMPLE, label)
cap = cv2.VideoCapture(0)
count = 1

frontData = [
[ 6.18816614e-01,  9.36991632e-01, -3.83637115e-07],
[ 0.57805794,  0.94313419 ,-0.02203732],
[ 0.53661525,  0.93871832 ,-0.05036191],
[ 0.4971875 ,  0.92944252 ,-0.07937451],
[ 0.46762967 , 0.89560771 ,-0.10999113],
[ 0.58766997  ,0.80343217 ,-0.05202853],
[ 0.53978252  ,0.77438569 ,-0.0760456 ],
[ 0.50237685  ,0.81324083 ,-0.09143702],
[ 0.47958815  ,0.85191685 ,-0.10228593],
[ 0.61273897 , 0.78064162 ,-0.05978587],
[ 0.56673312 , 0.67647696 ,-0.08282758],
[ 0.52877784 , 0.61883891 ,-0.1030723 ],
[ 0.49929228 , 0.56899476 ,-0.12137949],
[ 0.6367141  , 0.77880192 ,-0.06743598],
[ 0.60782295 , 0.67702293 ,-0.0992025 ],
[ 0.58191657 , 0.60108459 ,-0.12220884],
[ 0.55817086 , 0.53453904 ,-0.13887297],
[ 0.65907645 , 0.79647201 ,-0.07482422],
[ 0.64783442 , 0.74745983 ,-0.10262092],
[ 0.63315511 , 0.70945299 ,-0.11310611],
[ 0.61312377 , 0.66045773 ,-0.12178784],
]


# "o" 손동작의 값을 임시로 넣은 테스트용 데이터
        

joint = np.zeros((21, 3))
for i in range(len(frontData)):
    joint[i] = frontData[i][:]


# 각 랜드마크의 좌표 계산
v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
v = v2 - v1 # [20,3]
# 정규화
v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

# Get angle using arcos of dot product
angle = np.arccos(np.einsum('nt,nt->n',
    v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
    v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

angle = np.degrees(angle) # Convert radian to degree

# Inference gesture
data = np.array([angle], dtype=np.float32)
ret, results, neighbours, dist = knn.findNearest(data, 3)
idx = int(results[0][0])

print(gesture[idx])

ㅇ
